At inception.py, initial training of inceptionV3 takes place followed by the generation of H5 file.

In [ ]:
import tensorflow as tf

import keras
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import load_model
# demonstrate data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt

Load training data

In [ ]:
x_train = np.load("trainx.npy")
y_train = np.load("trainy.npy")

Change array values to the range 0 to 1.

In [ ]:
x_train = x_train.reshape(-1, 512, 512, 3)
x_train = x_train.astype('float32')
x_train = x_train / 255.

Image dimensions = 512x512x3 Number of classes (COVID,Normal,Viral) = 3

In [ ]:
input_shape = (512, 512, 3)
num_classes = 3
img_height = 512
img_width = 512
img_depth = 3

Initially, we will set a high learning rate for accelerated learning.

In [ ]:
batch_size = 4  # was 128
epochs = 16
learning_rate = 0.02

In [ ]:
# One-hot encoding for 3 classes.
y_train = keras.utils.to_categorical(y_train, num_classes)

Splitting input data (x_train, y_train) into 3 partitions: 95% for training, 5% for dev/validation.

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.05, random_state=17)

In [ ]:
model = keras.applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(512, 512, 3),
    pooling=max
)

include_top = False -> we are not adding fully connected layer at the top. Weights are set to imagenet weights initially. 
Transfer learning is applied here. The following layers with dropout are added to the model.

In [ ]:
x = model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(num_classes, activation='softmax')(x)
model = keras.models.Model(inputs=model.input, outputs=predictions)

Optimizer used: Adam with a learning rate of 0.02.

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=learning_rate),
              metrics=['accuracy'])
model.summary()

We will train the model now

In [ ]:
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_valid, y_valid))

# Save model in file
model.save("trained_inception_model.h5")